In [ ]:
!pip install -U scikit-learn scipy matplotlib
!pip install seaborn

In [ ]:
import pandas as pd 
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_parquet('../data/yellow_tripdata_2022-01.parquet')

In [ ]:
old_len = len(df)

In [ ]:
df.head()

In [ ]:
len(df.columns)

In [ ]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [ ]:
df.duration.mean()

In [ ]:
df.duration.std()

In [ ]:
old_len - ((df.duration >= 1) & (df.duration <= 60)).sum() 

In [ ]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [ ]:
df.duration.describe()

In [ ]:
df.isnull().mean()


In [ ]:
df

In [ ]:
df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].fillna(-1)

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].fillna(-1).astype('int')

In [ ]:
df[categorical] = df[categorical].astype('str')

In [ ]:
train_dicts = df[categorical].to_dict(orient='records')

In [ ]:
train_dicts[:10]

In [ ]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [ ]:
X_train.shape

In [ ]:
y_train = df.duration.values

In [ ]:
len(dv.feature_names_)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_train)

In [ ]:
mean_squared_error(y_train, y_pred, squared=False)

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
def read_data(filname):
    df = pd.read_parquet(filname)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df
    

In [ ]:
df_val = read_data('../data/yellow_tripdata_2022-02.parquet')

In [ ]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [ ]:
X_val = dv.transform(val_dicts)

In [ ]:
y_pred = lr.predict(X_val)

In [ ]:
y_val = df_val.duration.values

In [ ]:
mean_squared_error(y_val, y_pred, squared=False)